In [7]:
from bs4 import BeautifulSoup
import pandas as pd

#code for functions which will convert html data into pandas dataframe, as well as removing NULL values and formatting some of the fields.

def duration_to_seconds(duration_str):
    if type(duration_str) == type("a"):
        # Split the duration string by ':'
        parts = duration_str.split(':')
    
        # Convert the parts to integers
        parts = [int(part) for part in parts]
    
        # Calculate total seconds depending on the number of parts
        if len(parts) == 3:  # Format is hours:minutes:seconds
            return parts[0] * 3600 + parts[1] * 60 + parts[2]
        elif len(parts) == 2:  # Format is minutes:seconds
            return parts[0] * 60 + parts[1]
        else:
            return 0  # Return 0 or handle as an error for unexpected format
    return 0

def determine_video_type(row):
    if row['YouTube Music']: return 'Music'
    elif row['Video Duration'] < 62: return 'Shorts'
    else: return 'Video'
        
def dataFrameFormatter(df, monthShort):
    df = df[df['Date'].apply(lambda x: x.split(' ')[1] == monthShort if pd.notnull(x) else False)]
    df = df.drop_duplicates(subset='Title', keep='first')
    df = df[df['Video Duration'] != 0]
    df = df.reset_index(drop=True)
    df['Video Type'] = df.apply(determine_video_type, axis=1)
    df.drop(columns=['YouTube Music'], inplace=True)
    return df

def htmlToDataFrameConverter(monthShort,monthLong):
    file_path = 'HTML Files/'+monthLong+'.html'
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    dict = { "Date": [], "Title": [],"Title Length":[], "Time": [], "Video Duration": [], "YouTube Music": [] }
    
    soup = BeautifulSoup(html_content, "html.parser")    
    activities = soup.find_all("div", class_="GqCJpe u2cbPc LDk2Pd")
    # Extract data from each activity
    for activity in activities:
        # Extract date
        date = activity.find_previous("h2").text if activity.find_previous("h2") else None
        dict["Date"].append(date)
        # Extract video title
        title = activity.find("a", class_="l8sGWb").text if activity.find("a", class_="l8sGWb") else None
        dict["Title"].append(title)
        dict["Title Length"].append(len(title) if type(title) == type("a") else 0)
        # Extract time of activity
        time = activity.find("div", class_="H3Q9vf XTnvW").text.split("•")[0].strip() if activity.find("div", class_="H3Q9vf XTnvW") else None
        dict["Time"].append(time)
        # Extract video duration
        video_duration = activity.find("div", class_="bI9urf").text if activity.find("div", class_="bI9urf") else None
        dict["Video Duration"].append(duration_to_seconds(video_duration))
        # Check if it's YouTube Music content
        is_youtube_music = 'YouTube Music' in activity.text
        dict["YouTube Music"].append(is_youtube_music)
    
    return dataFrameFormatter(pd.DataFrame(dict),monthShort)
         




In [12]:
month_keywords = {"Oca":"January","Şub":"February","Mar":"March","Nis":"April","May":"May","Haz":"June",
                  "Tem":"July","Ağu":"August","Eyl":"September","Eki":"October","Kas":"November","Ara":"December"}
monthDFS = []
for a in month_keywords.keys():
    monthDFS.append(htmlToDataFrameConverter(a,month_keywords[a]).drop(["Title"],axis=1))
    print(month_keywords[a])

January
February
March
April
May
June
July
August
September
October
November
December


In [13]:
#code to combine monthly dataframes into one dataframe, as well as saving monthly data one by one.

month_names = ["January","February","March","April","May","June","July","August","September","October","November","December"]
for a in range(12):
    monthDFS[a].to_excel(month_names[a] + "_history.xlsx", index=False, engine='openpyxl')
    print(a)

combined_df = pd.concat(monthDFS, ignore_index=True)
combined_df.to_excel("2023_data_v1.xlsx",index=False,engine = 'openpyxl')

0
1
2
3
4
5
6
7
8
9
10
11


In [14]:
#code to extract the hour, month, and the day of the week the video is watched

import locale 
from datetime import datetime

locale.setlocale(locale.LC_TIME, 'tr_TR')

# Function to convert date string to month and weekday
def extract_month_weekday(date_str):
    # Parse the date string
    date_obj = datetime.strptime(date_str, '%d %b %Y')
    
    # Extract month and weekday (Python weekdays start from 0 (Monday) to 6 (Sunday))
    return date_obj.month, date_obj.weekday() + 1

# Function to extract hour from time
def extract_hour(time_str):
    hour = int(time_str.split(':')[0])
    return hour

combined_df['Month'], combined_df['Weekday'] = zip(*combined_df['Date'].apply(extract_month_weekday))
combined_df['Hour'] = combined_df['Time'].apply(extract_hour)

# Drop the 'Date' and 'Time' columns
combined_df.drop(columns=['Date', 'Time'], inplace=True)
combined_df.to_excel("2023_data_v2.xlsx",index=False,engine = 'openpyxl')